In [1]:
import os, sys
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns 

from os.path import join 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from IPython.display import display_html

import warnings
warnings.filterwarnings("ignore")

In [2]:
DataPath = r'D:\GitWork\titanic\data'
OutPath  = r'D:\GitWork\titanic\output'

f_abspath = join(DataPath, 'train.csv')
df_train = pd.read_csv(f_abspath)

f_abspath = join(DataPath, 'test.csv')
df_test = pd.read_csv(f_abspath)

df_data = df_train.append(df_test)
display(df_data.info())
display(df_data.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# For Sex
col_selected = ['PassengerId', 'Survived', 'Sex']
df_tmp = df_data[col_selected]

# Use pivot table
kwargs = { 'margins': True, 'margins_name': 'Total' }
pt = df_tmp.pivot_table(
    values='PassengerId',
    index='Sex',
    columns='Survived',
    aggfunc='count',
    **kwargs
)
pt.rename(columns={0.0:0, 1.0:1}, inplace=True)
pt['Rate_S1'] = (pt[1] / pt.Total).round(3)
display(pt)

# SexCode trandform
le_sex = LabelEncoder()
df_data['SexCode'] = le_sex.fit_transform(df_data.Sex)
print()
display(df_data.info())
display(df_data.head())

Survived,0,1,Total,Rate_S1
Sex,,,,
female,233,233,466,0.500
male,734,109,843,0.129
Total,967,342,1309,0.261



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  SexCode      1309 non-null   int32  
dtypes: float64(3), int32(1), int64(4), object(5)
memory usage: 138.1+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexCode
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [9]:
# Trainging
len_train = len(df_train)
ds_train = df_data[:len_train]

# If only use sex code to train, the survival prediction will be all 0. 
col_selected = ['PassengerId', 'Pclass', 'SexCode']
inps = ds_train[col_selected]
lbls = ds_train.Survived

display(inps.head())
display(inps.info())

model_rf = RandomForestClassifier(
    random_state = 2, 
    n_estimators = 250, 
    min_samples_split = 20, 
    oob_score = True
)
model_rf.fit(inps, lbls)
print('oob score :%f' %(model_rf.oob_score_))

,PassengerId,Pclass,SexCode
0,1,3,1
1,2,1,0
2,3,3,0
3,4,1,0
4,5,3,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  891 non-null    int64
 1   Pclass       891 non-null    int64
 2   SexCode      891 non-null    int32
dtypes: int32(1), int64(2)
memory usage: 24.4 KB


None

oob score :0.785634


In [10]:
# Test
ds_test = df_data[len_train:]

col_selected = ['PassengerId', 'Pclass', 'SexCode']
inps = ds_test[col_selected]

display(inps.head())
display(inps.info())

ds_test['Survived'] = model_rf.predict(inps)
display(ds_test.head())

,PassengerId,Pclass,SexCode
0,892,3,1
1,893,3,0
2,894,2,1
3,895,3,1
4,896,3,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Pclass       418 non-null    int64
 2   SexCode      418 non-null    int32
dtypes: int32(1), int64(2)
memory usage: 11.4 KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexCode
0,892,0.0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1
1,893,0.0,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,0.0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,0.0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1
4,896,0.0,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [13]:
col_selected = ['PassengerId', 'Survived', 'Sex']
ds_tmp = ds_test[col_selected]

pt = ds_tmp.pivot_table(
    values='PassengerId',
    index='Sex',
    columns='Survived',
    aggfunc='count',
    **kwargs
).fillna(0).astype(int)
pt.rename(columns={0.0:0, 1.0:1}, inplace=True)
pt['Rate_S1'] = (pt[1] / pt.Total).round(3)
display(pt)

Survived,0,1,Total,Rate_S1
Sex,,,,
female,72,80,152,0.526
male,266,0,266,0.000
Total,338,80,418,0.191


In [14]:
# Save predict result to xlsx
col_selected = ['PassengerId', 'Survived']
f_abspath = join(OutPath, 'Submission_v5.csv')
ds_test[col_selected].to_csv(f_abspath, index=False)

print('\ndone')


done
